In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x9eec390


In [2]:
ROOT.EnableImplicitMT(12)

In [3]:
ROOT.gInterpreter.Declare("#include <edm4hep/MCParticle.h>")
ROOT.gInterpreter.Declare("#include <edm4hep/utils/bit_utils.h>")

True

In [4]:
files = {
    "4f_ww_l": [],
    "4f_sw_sl": [],
    "4f_ww_h": [],
}

with open("data/locations/miniDSTs/processed-no-exc-higgs.txt", "r") as loc_file:
    for line in loc_file:
        for k in files:
            if k in line:
                files[k].append(line.strip())

In [5]:
df = {}
for k in files:
    df[k] = ROOT.RDataFrame("events", files[k])

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0xe2e72d0


Warning in <TClass::Init>: no dictionary for class edm4hep::Vector2i is available


In [6]:
ROOT.gStyle.SetPalette(ROOT.kBird)

In [7]:
for k, f in df.items():
    f = f.Define("p_vec_all", "Construct<ROOT::Math::XYZVector>(MCParticlesSkimmed.momentum.x, MCParticlesSkimmed.momentum.y, MCParticlesSkimmed.momentum.z)")
    f = f.Define("ovl_mask", "return Map(MCParticlesSkimmed.simulatorStatus, [] (const auto simStat) { return edm4hep::utils::checkBit(simStat, edm4hep::MCParticle::BITOverlay);})")
    f = f.Define("p_vec", "p_vec_all[ovl_mask]")
    f = f.Define("p_theta", "return Map(p_vec, [] (const auto& el) {return el.Theta();})")
    f = f.Define("p_pt", "return Map(p_vec, [] (const auto& el) {return el.Rho();})")
    f = f.Define("p_sintheta", "sin(p_theta)")
    f = f.Define("p_logpt", "log10(p_pt)")
    df[k] = f

In [8]:
h = {}
count = {}
for k, f in df.items():
    h[k] = f.Histo2D(("", f"{k};sin#theta;log_{{10}} p_{{T}}", 100, 0., 1., 100, -1., 2.), "p_sintheta", "p_logpt")
    count[k] = f.Count()

In [9]:
c = {}
for k in h:
    c[k] = ROOT.TCanvas()
    h[k].Draw("colz")
    c[k].Draw()
    c[k].SaveAs(f"plots/overlay/{k}.pdf")

Info in <TCanvas::Print>: pdf file plots/overlay/4f_ww_l.pdf has been created
Info in <TCanvas::Print>: pdf file plots/overlay/4f_sw_sl.pdf has been created
Info in <TCanvas::Print>: pdf file plots/overlay/4f_ww_h.pdf has been created


In [10]:
h_norm = {}
for k in h:
    h_t = h[k].Clone()
    h_t.Scale(1. / count[k].GetValue())
    h_norm[k] = h_t

In [11]:
h_sl_by_l = h_norm["4f_sw_sl"].Clone()
h_sl_by_l.Divide(h_norm["4f_ww_l"])

h_h_by_l = h_norm["4f_ww_h"].Clone()
h_h_by_l.Divide(h_norm["4f_ww_l"])

True

In [12]:
f = ROOT.TF1("f", "-2*x+0.2")

In [13]:
c_sl_by_l = ROOT.TCanvas()
h_sl_by_l.Draw("colz")
f.Draw("same")
c_sl_by_l.Draw()
c_sl_by_l.SetLogz()
h_sl_by_l.SetMinimum(0.1)
h_sl_by_l.SetMaximum(10.)

c_h_by_l = ROOT.TCanvas()
h_h_by_l.Draw("colz")
f.Draw("same")
c_h_by_l.Draw()
c_h_by_l.SetLogz()
h_h_by_l.SetMinimum(0.1)
h_h_by_l.SetMaximum(10.)